In [2]:
import configparser

these config files are hand-edited:

In [1]:
with open("prod.ini", "w") as prod, open("dev.ini", "w") as dev:
    prod.write("[Database]\n")
    prod.write("db_host=prod.mynetwork.com\n")
    prod.write("db_name=my_database\n")
    prod.write("\n[Server]\n")
    prod.write("port=8080\n")
    
    dev.write("[Database]\n")
    dev.write("db_host=dev.mynetwork.com\n")
    dev.write("db_name=my_database\n")
    dev.write("\n[Server]\n")
    dev.write("port=3000\n")

In [5]:
class Config:
    def __init__(self, env='dev'):
        print(f"Loading config from {env} file.")
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)
        self.db_host = config["Database"]["db_host"]
        self.db_name = config["Database"]["db_name"]
        self.port = config["Server"]["port"]

tipically the name of the environment is going to be a variable, for example, in could come from environment variables. Here we use just a string

In [7]:
config = Config("prod")

Loading config from prod file.


In [10]:
vars(config)

{'db_host': 'prod.mynetwork.com', 'db_name': 'my_database', 'port': '8080'}

we can have this module reloded wherever we need it and it will be compiled only once 

problems: 
    - inbuilt help doesn't show any useful info for Config
    - we needed to know all the sections in the .ini file and write them by hand

In [12]:
help(Config)

Help on class Config in module __main__:

class Config(builtins.object)
 |  Config(env='dev')
 |  
 |  Methods defined here:
 |  
 |  __init__(self, env='dev')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [13]:
class Config:
    def __init__(self, env='dev'):
        print(f"Loading config from {env} file.")
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)
        for section_name in config.sections():
            for key, value in config[section_name].items():
                setattr(self, key, value)
        
config = Config()       

Loading config from dev file.


In [14]:
vars(config)

{'db_host': 'dev.mynetwork.com', 'db_name': 'my_database', 'port': '3000'}

let's deal with sections, so we wouldn't have to flatten the structure

In [15]:
class Section:
    def __init__(self, name, item_dict):
        """
        name: str
            name of the section
        item_dict: dict
            dictionary of named (key) config values (value)
        """
        self.name = name
        for key, value in item_dict.items():
                setattr(self, key, value)

In [16]:
class Config:
    def __init__(self, env='dev'):
        print(f"Loading config from {env} file.")
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)
        for section_name in config.sections():
            section = Section(section_name, config[section_name])
            setattr(self, section_name.casefold(), section)

In [17]:
help(str.casefold)

Help on method_descriptor:

casefold(self, /)
    Return a version of the string suitable for caseless comparisons.



In [18]:
config = Config()       

Loading config from dev file.


In [19]:
vars(config)

{'database': <__main__.Section at 0x237bd048ec8>,
 'server': <__main__.Section at 0x237bd04c508>}

In [22]:
config.database.db_host

'dev.mynetwork.com'

built-in help() still doesn't have any information about database and server in the class, because they are instance variables. We will use metaclasses to make them class variables 

In [23]:
class SectionType(type):
    def __new__(cls, name, bases, cls_dict, section_name, items_dict):
        cls_dict['__doc__'] = f"Configs for {section_name} section."
        cls_dict['section_name'] = section_name
        for key, value in items_dict.items():
            cls_dict[key] = value
        return super().__new__(cls, name, bases, cls_dict)

we can create classes declaratively, but it won't be of use to us:

In [25]:
class DatabaseSection(metaclass=SectionType, section_name='database', 
                      items_dict={"db_host":"db host", "db_name":"db name"}):
    pass

In [26]:
vars(DatabaseSection)

mappingproxy({'__module__': '__main__',
              '__doc__': 'Configs for database section.',
              'section_name': 'database',
              'db_host': 'db host',
              'db_name': 'db name',
              '__dict__': <attribute '__dict__' of 'DatabaseSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DatabaseSection' objects>})

In [27]:
help(DatabaseSection)

Help on class DatabaseSection in module __main__:

class DatabaseSection(builtins.object)
 |  Configs for database section.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'db host'
 |  
 |  db_name = 'db name'
 |  
 |  section_name = 'database'



we will use functional approach to create classes:

In [32]:
MySection = SectionType("DBSection", (), {},  section_name='database', 
                      items_dict={"db_host":"db host", "db_name":"db name"})

In [33]:
MySection

__main__.DBSection

In [34]:
vars(MySection)

mappingproxy({'__doc__': 'Configs for database section.',
              'section_name': 'database',
              'db_host': 'db host',
              'db_name': 'db name',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'DBSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DBSection' objects>})

In [39]:
help(MySection)

Help on class DBSection in module __main__:

class DBSection(builtins.object)
 |  Configs for database section.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'db host'
 |  
 |  db_name = 'db name'
 |  
 |  section_name = 'database'



let's use it in our config metaclass. Attribute in config class will be the instances of Section class

In [40]:
class Configtype(type):
    def __new__(cls, name, bases, cls_dict, env):
        """
        env: str
           The environment we are loading for the config(e.g. prod, dev) 
        """
        cls_dict['__doc__'] = f"Config for {env}"
        cls_dict['env'] = env
        config = configparser.ConfigParser()
        file_name = f"{env}.ini"
        config.read(file_name)
        for section_name in config.sections():
            class_name = section_name.capitalize()
            class_attribute_name = section_name.casefold()
            section_items = config[section_name]
            bases = ()
            section_cls_dict = {}
            Section = SectionType(
                class_name, bases, section_cls_dict, section_name=section_name, items_dict = section_items
            )
            cls_dict[class_attribute_name] = Section
        return super().__new__(cls, name, bases, cls_dict)

In [41]:
class DevConfig(metaclass = Configtype, env='dev'):
    pass
class ProdConfig(metaclass = Configtype, env='prod'):
    pass

In [42]:
vars(DevConfig)

mappingproxy({'__module__': '__main__',
              '__doc__': 'Config for dev',
              'env': 'dev',
              'database': __main__.Database,
              'server': __main__.Server,
              '__dict__': <attribute '__dict__' of 'DevConfig' objects>,
              '__weakref__': <attribute '__weakref__' of 'DevConfig' objects>})

In [44]:
vars(DevConfig.database)

mappingproxy({'__doc__': 'Configs for Database section.',
              'section_name': 'Database',
              'db_host': 'dev.mynetwork.com',
              'db_name': 'my_database',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Database' objects>,
              '__weakref__': <attribute '__weakref__' of 'Database' objects>})

In [46]:
ProdConfig.server.port

'8080'

In [47]:
help(DevConfig)

Help on class DevConfig in module __main__:

class DevConfig(builtins.object)
 |  Config for dev
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  database = <class '__main__.Database'>
 |      Configs for Database section.
 |  
 |  env = 'dev'
 |  
 |  server = <class '__main__.Server'>
 |      Configs for Server section.



In [48]:
help(DevConfig.database )

Help on class Database in module __main__:

class Database(builtins.object)
 |  Configs for Database section.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'dev.mynetwork.com'
 |  
 |  db_name = 'my_database'
 |  
 |  section_name = 'Database'

